In [140]:
#Importamos las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import folium
from geopy.geocoders import Nominatim
import time
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier  # Ejemplo de modelo
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer



In [141]:
pp = pd.read_csv('C:\\Users\\jvazquez1\\Desktop\\proyecto_data\\Ingenias_proyecto\\datos\\pp.csv', sep=';', encoding='utf-8', low_memory=False)

In [142]:
# Leemos nuevamente los datos
pp.head()

,Row type,Advertiser,Advertiser code,Agent,Agent code,Invoice number,Product,Month,Year,Recipient,...,Exchange rate,Invoice amount gross home currency,Invoice amount net home currency,Contract amount home currency,Deal value home currency,Invoice ID,Deal Status,Invoice Type,Invoice Status,Discount amount
0,Contract transmission,S.A. La Nacion,A08009,ZenithOptimedia Group Argentina S.A,B02803,2019DC000780,Opcionales,9,2019,ZenithOptimedia Group Argentina S.A,...,5.986.232,251.44,120.75,156,11693.5,1708.0,Approved,Debit,Exported,0
1,Contract transmission,Arcor S.A.I.C.,A07784,ZenithOptimedia Group Argentina S.A,B02803,2019DC000683,Mogul,9,2019,ZenithOptimedia Group Argentina S.A,...,5.986.232,131.46,79.14,82,12365.66,1611.0,Approved,Debit,Exported,0
2,Contract transmission,Arcor S.A.I.C.,A07784,ZenithOptimedia Group Argentina S.A,B02803,2019DC000684,Mogul,9,2019,ZenithOptimedia Group Argentina S.A,...,5.986.232,57.2,34.42,35,12365.66,1612.0,Approved,Debit,Exported,0
3,Contract transmission,Arcor S.A.I.C.,A07784,ZenithOptimedia Group Argentina S.A,B02803,2019DC000685,Mogul,9,2019,ZenithOptimedia Group Argentina S.A,...,5.986.232,39.9,24.01,25,12365.66,1613.0,Approved,Debit,Exported,0
4,Contract transmission,Arcor S.A.I.C.,A07784,ZenithOptimedia Group Argentina S.A,B02803,2019DC000686,Mogul,9,2019,ZenithOptimedia Group Argentina S.A,...,5.986.232,146.46,88.23,91,12365.66,1614.0,Approved,Debit,Exported,0


In [143]:
#Antes que nada, chequeemos nuevamente que eliminamos todos los valores faltantes.
pp.isnull().sum()

Row type                                  0
Advertiser                                0
Advertiser code                           0
Agent                                     0
Agent code                                0
Invoice number                          947
Product                                   0
Month                                     0
Year                                      0
Recipient                                 0
Contract                                  0
Invoice amount gross                      0
 Commission amount                        0
Invoice amount net                        0
Channel                                   0
Channel code                              0
Item start date                           0
Item end date                             0
SAP code                                  0
Contract status                        3723
Invoice amount total                      0
Invoice date                            947
Invoice batch number            

¿Qué tipo de problemas estamos tratando de resolver? 
Desarrollar un modelo predictivo que permita identificar: 
*patrones de gasto publicitario, 
*tendencias estacionales 
*efectividad de los canales de publicidad 
en función de las variables disponibles en el dataset.

Variables:
Invoice amount gross= cantidad de dinero gastado en publicidad
Channel= canales que más se usan
Month y year = fechas que más se publicita

REGRESIÓN

In [196]:
# Elimino aquellas variables que no quiero incluir en el modelo y las guardo en x.
x = pp.drop(['Row type', 'Advertiser code','Agent code', 'Invoice number', 'Recipient', 'Contract', 'Channel code', 'Item start date', 'Item end date', 'SAP code', 'Invoice amount total', 'Invoice date', 'Invoice batch number', 'Invoice currency', 'Deal', 'Deal currency', 'Deal value', 'Invoice holding company', 'Sales holding company', 'Billing scenario', 'Incentive discount value', 'Exchange rate', 'Invoice amount gross home currency', 'Invoice amount net home currency', 'Contract amount home currency', 'Deal value home currency', 'Invoice ID', 'Deal Status', 'Invoice Type', 'Invoice Status','Discount amount'], 
                  axis=1)

In [197]:
x.head()

,Advertiser,Agent,Product,Month,Year,Invoice amount gross,Commission amount,Invoice amount net,Channel,Contract status,Rep,Unnamed: 44,Unnamed: 45,Unnamed: 46,Incentive discount %
0,S.A. La Nacion,ZenithOptimedia Group Argentina S.A,Opcionales,9,2019,9345,2116.64,7228.36,DLA Home & Health Argentina,Approved,Soledad,NaN,NaN,NaN,9.0
1,Arcor S.A.I.C.,ZenithOptimedia Group Argentina S.A,Mogul,9,2019,4883.76,146.51,4737.25,DLA Discovery Argentina,Approved,Soledad,NaN,NaN,NaN,0.0
2,Arcor S.A.I.C.,ZenithOptimedia Group Argentina S.A,Mogul,9,2019,2124.2,63.73,2060.47,DLA Animal Planet Argentina,Approved,Soledad,NaN,NaN,NaN,0.0
3,Arcor S.A.I.C.,ZenithOptimedia Group Argentina S.A,Mogul,9,2019,1482,44.46,1437.54,DLA TLC Argentina,Approved,Soledad,NaN,NaN,NaN,0.0
4,Arcor S.A.I.C.,ZenithOptimedia Group Argentina S.A,Mogul,9,2019,5445,163.35,5281.65,DLA Discovery ID Argentina,Approved,Soledad,NaN,NaN,NaN,0.0


In [198]:
lista_atributos = x.columns

In [199]:
# Ahora selecciono las etiquetas y las guardo en y.
y = pp['Invoice amount gross']

In [200]:
x, y = np.array(x), np.array(y)

In [201]:
x

array([['S.A. La Nacion', 'ZenithOptimedia Group Argentina S.A',
        'Opcionales', ..., nan, nan, 9.0],
       ['Arcor S.A.I.C.', 'ZenithOptimedia Group Argentina S.A', 'Mogul',
        ..., nan, nan, 0.0],
       ['Arcor S.A.I.C.', 'ZenithOptimedia Group Argentina S.A', 'Mogul',
        ..., nan, nan, 0.0],
       ...,
       ['Inc S.A.', 'Groupm Argentina Trading S.A.', 'Carrefour', ...,
        nan, nan, 0.0],
       ['Bagley Argentina S.A.', 'MMS Comunicaciones Arg. S.R.L.', 'BYN',
        ..., nan, nan, 0.0],
       ['Bagley Argentina S.A.', 'MMS Comunicaciones Arg. S.R.L.', 'BYN',
        ..., nan, nan, 0.0]], dtype=object)

In [202]:
y[:10]

array(['9345', '4883.76', '2124.2', '1482', '5445', '1294.28', '8519.6',
       '129600', '129600', '4416'], dtype=object)

Separando Train / Test

In [203]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, 
                                                    random_state=42)

In [204]:
x.shape

(34762, 15)

In [205]:
X_train.shape

(27809, 15)

In [206]:
X_test.shape

(6953, 15)

Random Forest

In [207]:
# Inicializo el modelo
regresor = RandomForestRegressor(criterion='absolute_error', random_state=42)

In [208]:
# Entreno el modelo
regresor.fit(X_train, y_train)

ValueError: could not convert string to float: 'Renault Argentina'

In [209]:
regresor.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'absolute_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [210]:
# Entrenar el modelo con los datos de entrenamiento preprocesados
model.fit(X_train, y_train)

# Predigo los valores para el set de testeo usando el modelo ya entrenado
y_pred = model.predict(X_test)

ValueError: Specifying the columns using strings is only supported for pandas DataFrames

In [166]:
# Calculo el error medio absoluto
mean_absolute_error(y_test, y_pred)

NameError: name 'y_pred' is not defined

### Support Vector Machine 

In [211]:
from sklearn.svm import SVR

In [212]:
# Inicializo el modelo
regresor_svr = SVR(C=1.0, epsilon=0.2)

In [213]:
# Entreno el modelo
regresor_svr.fit(X_train, y_train)

ValueError: could not convert string to float: 'Renault Argentina'

In [214]:
regresor_svr.get_params()

{'C': 1.0,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.2,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [215]:
# Predigo los valores para el set de testeo
y_pred_svr = regresor_svr.predict(X_test)

NotFittedError: This SVR instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [216]:
# Calculo el error medio absoluto
mean_absolute_error(y_test, y_pred_svr)

NameError: name 'y_pred_svr' is not defined